## 객체를 제너릭하게 구성하려면 @classmethod를 통한 다형성 활용하라

**파이썬은 객체뿐만 아니라 클래스도 다형성 지원**

- 다형성을 사용하면 계층을 이루는 여러 클래스가 자신에게 맞는 유일한 메서드 버전 구현

   - 서로 다른 기능을 제공

In [1]:
# 맵리듀스 구현
# 입력 데이터를 표현할 수 있는 공동 클래스가 필요

#read메서드가 들어있는 공통 클래스

class InputData:
    def read(self):
        raise NotImplementedError

In [2]:
#하위 클래스 만들어서 파일 읽기\

class PathInputData(InputData):
    def __init__(self, path):
        super().__init__()
        self.path = path

    def read(self):
        with open(self.path) as f:
            return f.read()

In [3]:
#입력 데이터를 소비하는 공통 방법을 제공하는 추상 인터페이스
class Worker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None

    def map(self):
        raise NotImplementedError
    
    def reduce(self, other):
        raise NotImplementedError

        


In [4]:
#각 객체를 만들고 맵리듀스를 조합롭게 실행하는 법
# 도우미 함수를 활용해서 직접 만들고 연결
import os

def generate_inputs(data_dir):
    for name in os.listdir(data_dir):
        yield PathInputData(os.path.join(data_dir,name))

"""위의 함수를 통해서 InputData 인스턴스를 사용하는LineCountWorker 인스턴스를 만든다.
"""

def create_workers(input_list):
    workers = []
    for input_data in input_list:
        workers.append(LineCountWorker(input_data))
    return workers


In [5]:
"""
Worker인스턴스의 map단계를 여러 스레드에 공급해서 실행 가능
"""

from threading import Thread

def execute(workers):
    threads = [Thread(target=w.map) for w in workers]
    for thread in threads: thread.start()
    for thread in threads: thread.join()

    first, *rest = workers
    for worker in rest:
        first.reduce(worker)
    return first.result

def mapreduce(data_dir):
    inputs = generate_inputs(data_dir)
    workers = create_workers(inputs)
    return execute(workers)

In [6]:
#클래스 메서드 다형성을 사용하여 제너릭한 방식으로 객체 구성

"""
클래스 메서드, 멥리듀스에 사용했던 클래스에 적용하기
@classmethod적용된 클래스 메서드가 공통 인터페이스를 통해서 새로운 InputData 인스턴스 생성
"""

class GenericInputData:
    def read(self):
        raise NotImplementedError

    @classmethod
    def generate_inputs(cls, config):
        raise NotImplementedError

In [11]:

"""
generate_input는 GenericInputData의 구체적인 하위 클래스가 객체 생성
""" 
class PathInputData(InputData):
    def __init__(self, path):
        super().__init__()
        self.path = path

    def read(self):
        with open(self.path) as f:
            return f.read()

    @classmethod
    def generate_inputs(cls, config):
        data_dir = config['data_dir']
        for name in os.listdir(data_dir):
            yield cls(os.path.join(data_dir, name))






In [12]:
"""
위와 같은 방식으로 GenericWork클래스 안에 create_workers 도우미 메서드 추가
도우마 메서드는 GenericInputData의 하위타입이어야하는 input_class 파라미터
"""

class GenericWorker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None

    def map(self):
        raise NotImplementedError

    def reduce(self, other):
        raise NotImplementedError

    @classmethod
    def create_workers(cls, input_class, config):
        workers = []
        for input_data in input_class.generate_inputs(config):
            workers.append(cls(input_data))
        return workers


In [15]:
# mapreduce함수가 create_workers를 호출하게 변경해서 mapreduce를 완전한 제너릭 함수
class LineCountWorker(GenericWorker):

    def mapreduce(worker_class, input_class, config):
        workers = worker_class.create_workers(input_class, config)
        return execute(workers)

#config = {'data_dir': }
#result = mapreduce(LineCountWorker, PathInputData, config)
#print(f'총 {result} 줄이 있습니다.')
